# Dataloader Class

In [1]:
# Imports
import os
from warnings import warn
import json
import numpy as np
import pandas as pd
from datetime import datetime
import math
from math import floor, sqrt
from pandarallel import pandarallel
import time

In [2]:
# Whether to using multiprocessing. Turn on and off by setting the value of using_parallel

using_parallel = False

if using_parallel:
    pandarallel.initialize()

In [3]:
# Util function for FOV check -- by Rajay

# pos_player (position of player) and pos_object (position of object) are triples (x, y, z)
# object triples are all integers. Minecraft block coordinate is a corner with the least value.
# Assume all positions are given in relative coordinates from some chosen origin.
# Pitch and Yaw are given by Minecraft and are in degrees. pitch_player must be between -90 and +90 degrees and yaw_player must be between -180 and 180 degrees.
# South: +Z, North: -Z, East: +X, West: -X
# Pitch: Increases from 0 to +180 degrees when sweeping from South, through West, to North
# Pitch: Decreases from 0 to -180 degrees when sweeping from South, through East, to North
# Yaw: Increases from 0 to +90 degrees when looking toward feet
# Yaw: Decreases from 0 to -90 degrees when looking toward above
# distance_cutoff: torch viewing distance; Anything closer than this distance is visible. If distance_cutoff is less than or equal to zero, then it is not taken into account.
def CanSee(pos_player, pos_object, pitch_player, yaw_player, distance_cutoff = -1, debug=False):
    FOV_PITCH = 63.5 # Degrees
    FOV_YAW = 101 # Degrees
    EYE_HEIGHT = 1.62 # eye height according to https://minecraft.gamepedia.com/Tutorials/Navigation#X.2C_Y.2C_and_Z_coordinates

    result = False
    center_angel_yaw_deg, center_angel_pitch_deg = 0, 0
    for i in [0,0.5,1]:
        for j in [0,0.5,1]:
            for k in [0,0.5,1]:
                xDist = pos_player[0] - (pos_object[0]+i)
                yDist = (pos_player[1]+EYE_HEIGHT) - (pos_object[1]+j)
                zDist = pos_player[2] - (pos_object[2]+k)
                # 3-d distance
                d3 = math.sqrt(math.pow(xDist,2) + math.pow(yDist,2) + math.pow(zDist,2))
                # if distance_cutoff is positive, then if distance to object is too far, it can't be seen
                if (distance_cutoff > 0 and d3 > distance_cutoff):
                    continue
                # on-plane distance (h=horizontal)
                dh = math.sqrt(math.pow(xDist,2) + math.pow(zDist,2))
                # vertical distance (v=vertical)
                dv = math.sqrt(math.pow(yDist,2) + math.pow(dh,2))
                angle_yaw_deg = RestrictAngle(math.atan2(zDist,xDist)*180/math.pi + 90)
                angle_pitch_deg = math.atan2(yDist,dh)*180/math.pi
                if i == 0.5 and j == 0.5 and k == 0.5:
                    center_angel_yaw_deg, center_angel_pitch_deg = angle_yaw_deg, angle_pitch_deg
#                 print((angle_yaw_deg, angle_pitch_deg)) # testing
                isInPitch = IsAngleBetween(angle_pitch_deg, pitch_player-FOV_PITCH/2, pitch_player+FOV_PITCH/2)
                isInYaw   = IsAngleBetween(angle_yaw_deg, yaw_player-FOV_YAW/2, yaw_player+FOV_YAW/2)
                if debug:
                    print((angle_yaw_deg, angle_pitch_deg, isInPitch, isInYaw)) # testing
                result += isInPitch and isInYaw
                
    return bool(result), center_angel_yaw_deg, center_angel_pitch_deg   

def RestrictAngle(angle):
    if (angle > 180):
        return angle - 360
    elif (angle < -180):
        return angle + 360
    else:
        return angle

def IsAngleBetween(angleToCheck,minAngle,maxAngle):
    if (maxAngle > 180):
#         print("Is " + str(angleToCheck) + " between " + str(minAngle) + " and 180 or greater than -180 and less than " + str(RestrictAngle(maxAngle)))
        return ((minAngle <= angleToCheck and (angleToCheck <= 180)) or (minAngle >= -180 and (angleToCheck <= RestrictAngle(maxAngle))))
    else:
#         print("Is " + str(angleToCheck) + " between " + str(minAngle) + " and " + str(maxAngle))
        return (minAngle <= angleToCheck and angleToCheck <= maxAngle)


In [88]:
# Dataloader class

class DataLoader:
    def __init__(self, verbose=True):
        # Raw data
        self.raw_df = None                 # Dataframe from raw data
        self.room_df = None             # Map location dataframe
        self.victim_df = None          # Victim location dataframe
        self.door_df = None            # Door location dataframe
        
        # Intermediat data
        self.obs_df = None             # Observation dataframe
        self.event_df = None           # Event dataframe
        self.triage_df = None          # Triage event dataframe
        self.conflict_victim_df = None # Dataframe containing info about occurence of different victim_id appearing in proc_df and triage_df
        
        # Processed data
        self.proc_df = None            # Stores dataframe processed by specific routines
        
        # Other relevant data fields and attributes
        self.player_names = None       # Tuple of player names
        self.player_role = None        # Dictionary mapping from player name to player's role
        self.datestr_format = "%Y-%m-%dT%H:%M:%S.%f"   # timestamp format in the raw data
        self.max_victims_per_room = None   # Max number of victims per room. Will calculate automatically
        self.max_portal_in_FOV = 3     # The max number of portals in FOV to be included in the generated data
        self.max_triage_dist = 5       # Maximum distance within which player can be considered in triage position
        self.victim_time = {}          # Victim type (color) to the number of seconds to successfuly triage
        self.num_per_sec = 1           # Number of entries to select per second. Default to 1
        self.sensor_events = set()     # Set of events that will be recognized as sensor "beeping"
        
        # Offset to ADD to raw x, y, z position values
        # For now, will calculate automatically by finding minimum for each coordinate globally in the dataframe
        self.x_offset, self.y_offset, self.z_offset = None, None, None
        self.x_max, self.y_max, self.z_max = None, None, None
        
        # Verbosity. Whether to print progress and/or warnings. 1 to print, 0 to shut up.
        self.verbose = verbose
    
    def load_csv(self, path, file_name):
        # thousands="," here so that numbers are of type float64
        self.raw_df = pd.read_csv(os.path.join(path, file_name), thousands=",")[['message']]   
        # Initial processing
        self._init_df()
        
    def load_json(self, path, file_name):
        self.raw_df = pd.read_json(os.path.join(path, file_name), lines=True)[['message']]
        # Initial processing
        self._init_df()
        
    def _init_df(self):
        # If no player name and roles has been specified, default treat all names found in the data as Triager
#         assert self.player_names is not None and self.player_role is not None, "Please first set the player names and corresponding roles"

        # Clean up data. 
        # Convert message content from string to dictionary using Json
        self.raw_df['content'] = self.raw_df['message'].apply(lambda entry: json.loads(entry))
        # List dictionary keys and drop any columns that have empty keys
        self.raw_df['content_keys'] = self.raw_df['content'].apply(lambda entry: tuple(entry.keys()))
        empty_indices = list(self.raw_df.loc[self.raw_df['content_keys'] == ()].index)
        if len(empty_indices) > 0:
            self.raw_df = self.raw_df.drop(labels=empty_indices, axis=0)
            # Reset index
            self.raw_df = self.raw_df.reset_index().drop('index', axis=1)
        # Drop entries if 'header' does not appear as one of the content keys
        has_header = self.raw_df['content_keys'].apply(lambda entry: 'header' in entry)
        no_header_index = list(has_header[has_header == False].index)
        if len(no_header_index) > 0:
            self.raw_df = self.raw_df.drop(labels=no_header_index, axis=0)
            # Reset index
            self.raw_df = self.raw_df.reset_index().drop('index', axis=1)
            
        # Extract timestamp string
        self.raw_df['@timestamp'] = self.raw_df['content'].apply(lambda entry: entry['header']['timestamp'])
        # Extract message type and publish to another column 'message_type'
        self.raw_df['message_type'] = self.raw_df['content'].apply(lambda entry: entry['header']['message_type'])
        
        # Separate out event df 
        self.event_df = self.raw_df.loc[self.raw_df['message_type'] == 'event']
        self.event_df = self.event_df.reset_index().drop('index', axis=1)
        # Extract message for event df
        self.event_df['msg'] = self.event_df['content'].apply(lambda entry: entry['msg'])
        self.event_df['data'] = self.event_df['content'].apply(lambda entry: entry['data'] if 'data' in entry.keys() else 'None')
        # Publish event subtype to another column
        self.event_df['subtype'] = self.event_df['msg'].apply(lambda entry: entry['sub_type'])
        
        # Find Event:MissionState subtype event timestamp and use that to chop off data irrelevant to the mission
        missionstate_df = self.event_df.loc[self.event_df['subtype'] == 'Event:MissionState']
        if missionstate_df.shape[0] == 0 and self.verbose:
            warn("\nNo Event:MissionState event found! All raw data entries are therefore taken valid\n")
        else:
            missionstate_df['mission_state'] = missionstate_df['content'].apply(lambda entry: entry['data']['mission_state'].upper())
            if 'START' in set(missionstate_df['mission_state'].unique()):
                # Take the earlies entry
                start_timestamp = missionstate_df.loc[missionstate_df['mission_state'] == 'START'].iloc[-1]['@timestamp']
                # Chopp off any data that registers befor this timestamp
                self.raw_df = self.raw_df.loc[pd.to_datetime(self.raw_df['@timestamp']) >= start_timestamp]
                self.event_df = self.event_df.loc[pd.to_datetime(self.event_df['@timestamp']) >= start_timestamp]
                self.raw_df = self.raw_df.reset_index().drop('index', axis=1)
                self.event_df = self.event_df.reset_index().drop('index', axis=1)
            elif self.verbose:
                warn("\nNo 'mission_state == START' event found. Entries early in time will not be chopped off\n")
            if 'STOP' in set(missionstate_df['mission_state'].unique()):
                # Take the latest entry
                stop_timestamp = missionstate_df.loc[missionstate_df['mission_state'] == 'STOP'].iloc[0]['@timestamp']
                # Chopp off any data that registers befor this timestamp
                self.raw_df = self.raw_df.loc[pd.to_datetime(self.raw_df['@timestamp']) <= stop_timestamp]
                self.event_df = self.event_df.loc[pd.to_datetime(self.event_df['@timestamp']) <= stop_timestamp]
                self.raw_df = self.raw_df.reset_index().drop('index', axis=1)
                self.event_df = self.event_df.reset_index().drop('index', axis=1)
            elif self.verbose:
                warn("\nNo 'mission_state == STOP' event found. Entries late in time will not be chopped off\n")
            
        # Extract triaging events to triage df
        self.triage_df = self.event_df.loc[self.event_df['subtype'] == 'Event:Triage']       
        self.triage_df[['player_ID', 'triage_state', 'victim_x', 'victim_y', 'victim_z']] = \
            self.triage_df.apply(lambda row: [row['data']['playername'], 
                                              row['data']['triage_state'],
                                              row['data']['victim_x'],
                                              row['data']['victim_y'],
                                              row['data']['victim_z']], axis=1, result_type='expand')
        # Only select entries with player ID pre-specified
        if self.player_names is not None:
            self.triage_df = self.triage_df.loc[self.triage_df['player_ID'].isin(self.player_names)]
        # Otherwise, set default player names and roles
        else:
            self.player_names = tuple(set(self.triage_df['player_ID'].unique()))
            self.player_role = {n: 'Triager' for n in self.player_names}
        # Reset index and drop redundant columns
        self.triage_df = self.triage_df.reset_index().drop(['index', 'message', 'content', 'content_keys', 'message_type', 'msg', 'data', 'subtype'], axis=1) 
        # Sort dataframe so that timestamp in ascending order
        self.triage_df = self.triage_df.sort_values(by='@timestamp', axis=0).reset_index().drop('index', axis=1)
        
        # Seperate out observation dataframe
        self.obs_df = self.raw_df.loc[self.raw_df['message_type'] == 'observation']
        self.obs_df = self.obs_df.reset_index().drop('index', axis=1)
        self.obs_df['data'] = self.obs_df.apply(lambda row: row['content']['data'], axis=1)
        # Get rid of scoreboard observation data entry
        self.obs_df['data_keys'] = self.obs_df['data'].apply(lambda entry: tuple(entry.keys()))
        self.obs_df = self.obs_df.loc[self.obs_df['data_keys'] != ('scoreboard',)]
        self.obs_df = self.obs_df.reset_index().drop(['index', 'data_keys'], axis=1)
        
        # Set up proc_df
        self.proc_df = self.obs_df.apply(lambda row: [row['@timestamp'], row['data']['name'],
                                                      row['data']['motion_x'], row['data']['motion_y'], row['data']['motion_z'], 
                                                      row['data']['x'], row['data']['y'], row['data']['z'], 
                                                      row['data']['pitch'], row['data']['yaw']], axis=1, result_type='expand')
        # Rename column names
        self.proc_df.columns = ['@timestamp', 'player_ID', 'motion_x', 'motion_y', 'motion_z', 'x', 'y', 'z', 'pitch', 'yaw']
        # Only select entries with player ID pre-specified
        self.proc_df = self.proc_df.loc[self.proc_df['player_ID'].isin(self.player_names)]
        # Reset index 
        self.proc_df = self.proc_df.reset_index().drop('index', axis=1)
    
    def load_map(self, path, room_csv_name, door_csv_name, victim_csv_name):
        self.room_df = pd.read_csv(os.path.join(path, room_csv_name))
        self.door_df = pd.read_csv(os.path.join(path, door_csv_name))
        self.victim_df = pd.read_csv(os.path.join(path, victim_csv_name))
        # Rename column names
        self.room_df.columns = ['RoomID', 'x0', 'z0', 'x1', 'z1']
        self.door_df.columns = ['Index', 'x0', 'z0', 'x1', 'z1', 'Room0', 'Room1'] 
        self.victim_df.columns = ['Index', 'RoomID', 'Color', 'x', 'y', 'z']
        
        # Calculate a mid point for the doors/portals as FOV calculation's reference
        self.door_df[['x_mid', 'z_mid']] = self.door_df.apply(lambda row: [(row['x0'] + row['x1']) / 2, (row['z0'] + row['z1']) / 2], 
                                                              axis=1, result_type='expand')
        
    def set_loc_offsets(self, x_offset, y_offset, z_offset):
        self.x_offset = x_offset
        self.y_offset = y_offset
        self.z_offset = z_offset
        
    def set_player_role(self, player_role_dict):
        assert isinstance(player_role_dict, dict)
        self.player_role = player_role_dict
        self.player_names = set(player_role_dict.keys())
        
    def set_victim_triage_time_span(self, victim_time_span_dict):
        assert isinstance(victim_time_span_dict, dict)
        assert set(self.victim_df['Color']) == set(victim_time_span_dict.keys())
        self.victim_time = victim_time_span_dict
        
    def set_max_triage_distance(self, max_triage_distance):
        self.max_triage_dist = max_triage_distance
        
    def set_max_portals_in_FOV(self, num_portals):
        self.max_portal_in_FOV = num_portals
        
    def set_num_entries_per_sec(self, num_per_sec):
        # Set the number of entries to select per second
        self.num_per_sec = num_per_sec
        
    def add_sensor_event_subtype(self, event):
        # Add event subtype identifier string which will be picked up and treated as a sensor "beeping"
        self.sensor_events.add(event)
        
    def process(self):
        # Process data using subroutines. processed data in self.proc_df
        # Need to all methods above before calling this method, otherwise requisite data not complete
        # All subroutines operate on self.proc_df
        # Process
        self._df_by_sec()                    # Extract data entries with a coarser time granularity
        self._adjust_loc()                   # Apply offset to all coordinates
        self._check_victim_in_room()         # SANITY CHECK: whether the victims are indeed located in the room that the data claimed they are
        self._find_room_in()                 # Find which room the player is located in
        self._find_portal_in()               # Find which portal the player is located in, if applicable
        self._find_victims_in_room()         # Find which victims are located in the same room as the player. Grab victims from both rooms if the player is located in a portal
        self._set_role()                     # Set player roles (default to TRIAGER)
        self._victim_in_FOV()                # For each victim, determine if it is in the Field of View of the player
        self._victim_in_triage_position()    # For each victim, determine if it is in the player's triaging position (in FOV + close enough)

        self._victim_in_cross_hair()         # Find the one victim that is closest to the visual center in the player's field of view
        self._portal_in_FOV()                # Find which portals are in the field of view of the player. Sorted in ascending order by the portal's distance to the player
        self._identify_triage_id()           # Identify the triage events and the indices of their occurences in the main dataframe
        self._set_triage_in_progress()       # Set the triage state based on the reported triaging events
        self._update_victim_color()          # Update victim colors. E.g. victim turned white from green when they are triaged
        self._set_sensor_state()             # Set sensor state
        
        
#     def batch_process_json(self, path):
#         # Attempt to batch process all JSON files in the given path, and produce processed csv file with "processed_" prefix in the file name
#         #     under the subdirectory name "processed"
#         for file_name in os.listdir(path):
#             if file_name.endswith(".json"):
#                 try:
#                     print("\n\n---------------- Processing file {} ----------------\n".format(file_name))
#                     self.load_json(path, file_name)
#                     self.process()
#                     self.save_csv(os.path.join(path, 'processed'), "processed_" + os.path.splitext(file_name)[0] + '.csv')
#                     print("\n---------------- File saved ----------------\n\n")
#                 except Exception as err:
#                     warn("CANNOT PROCESS JSON FILE: {}. Encountered the following exception: \n\n{}".format(file_name, err))
                    
        
    def save_csv(self, path, filename):
        # Save the processed data, self.proc_df, to a csv file
        # If directory not exists, create the directory
        if not os.path.exists(path):
            os.makedirs(path)
        self.proc_df.to_csv(os.path.join(path, filename))
        
    def _df_by_sec(self):
        # Take a coarser time granularity to select a fixed number of entries per second. 
        # Update self.player2df
        tic = time.perf_counter()
        
        player_group = self.proc_df.groupby(by="player_ID")
        player2df = {n: player_group.get_group(n) for n in self.player_names}
        
        player2df_sec = {}
        for n in self.player_names:
            p_df = player2df[n]
            # Reset index by datetime
            p_df.index = pd.to_datetime(p_df['@timestamp'],format=self.datestr_format)
            # Add a new column to track which microsecond group the entries belong to.
            # To support selecting multiple entries per second which is spread equally in time
            p_df['ms_group'] = p_df.index.microsecond // (1000000 // self.num_per_sec)
            # Groupby seconds and microsecond group
            g = p_df.groupby(by=[p_df.index.hour, p_df.index.minute, p_df.index.second, p_df['ms_group']])
            # Take last of each group, which is the earliest entry per microsecond group
            p_df = g.last()
            # Reset index. argument drop=True to drop the current datetime index
            p_df.reset_index(drop=True, inplace=True)
            player2df_sec[n] = p_df
            
        player2df = player2df_sec
        # Update self.proc_df
        self.proc_df = pd.concat(list(player2df.values()), axis=0)
        self.proc_df.reset_index(inplace=True)
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 1: df_by_sec() completed. Elapsed time: {:.5f}".format(toc - tic))
        
    def _adjust_loc(self):
        # Adjust proc_df loc
        tic = time.perf_counter()
        
        # If offset not set, set it automatically by finding the minimum of each coordinates
        if self.x_offset is None or self.y_offset is None or self.z_offset is None:
            self.set_loc_offsets(-np.floor(self.proc_df["x"].min()), -np.floor(self.proc_df["y"].min()), -np.floor(self.proc_df["z"].min()))
        
        self.proc_df["x"] = self.proc_df["x"].map(lambda v: np.floor(v) + self.x_offset)
        self.proc_df["y"] = self.proc_df["y"].map(lambda v: np.floor(v) + self.y_offset)
        self.proc_df["z"] = self.proc_df["z"].map(lambda v: np.floor(v) + self.z_offset)
        
        assert self.proc_df["x"].min() >= 0, self.proc_df["x"].min()
        assert self.proc_df["y"].min() >= 0, self.proc_df["y"].min()
        assert self.proc_df["z"].min() >= 0, self.proc_df["z"].min()
            
        self.x_max = self.proc_df["x"].max()
        self.y_max = self.proc_df["y"].max()
        self.z_max = self.proc_df["z"].max()
        
        # Adjust room loc
        self.room_df[["x0", "x1"]] = self.room_df[["x0", "x1"]].apply(lambda v: np.floor(v) + self.x_offset)
        self.room_df[["z0", "z1"]] = self.room_df[["z0", "z1"]].apply(lambda v: np.floor(v) + self.z_offset)
        # Take max/min for x/z and insert new columns
        self.room_df["x_max"], self.room_df["x_min"] = self.room_df[["x0", "x1"]].max(axis=1), self.room_df[["x0", "x1"]].min(axis=1)
        self.room_df["z_max"], self.room_df["z_min"] = self.room_df[["z0", "z1"]].max(axis=1), self.room_df[["z0", "z1"]].min(axis=1)
        # Drop original coordinates columns
        self.room_df.drop(labels=['x0', 'x1', 'z0', 'z1'], axis=1, inplace=True)
        
        # Adjust door loc
        self.door_df[['x0', 'x1']] = self.door_df[["x0", "x1"]].apply(lambda v: np.floor(v) + self.x_offset)
        self.door_df[["z0", "z1"]] = self.door_df[["z0", "z1"]].apply(lambda v: np.floor(v) + self.z_offset)
        
        # Adjust victim loc
        # Use floor() to round
        self.victim_df['x'] = self.victim_df['x'].map(lambda v: np.floor(v) + self.x_offset)
        self.victim_df['y'] = self.victim_df['y'].map(lambda v: np.floor(v) + self.y_offset)
        self.victim_df['z'] = self.victim_df['z'].map(lambda v: np.floor(v) + self.z_offset)
        
        # Adjust triage dataframe loc
        self.triage_df['victim_x'] = self.triage_df['victim_x'].map(lambda v: np.floor(v) + self.x_offset)
        self.triage_df['victim_y'] = self.triage_df['victim_y'].map(lambda v: np.floor(v) + self.y_offset)
        self.triage_df['victim_z'] = self.triage_df['victim_z'].map(lambda v: np.floor(v) + self.z_offset)
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 2: adjust_loc() completed. Elapsed time: {:.5f}".format(toc - tic))
        
        
    def _check_victim_in_room(self):
        # Check that after adjusting locations, each victim is indeed within the room that it labels in the dataframe
        #rg = self.room_df.groupbyby(by=)
        tic = time.perf_counter()
        
        rg = self.room_df.groupby(by='RoomID')
        vg = self.victim_df.groupby(by='RoomID')
        
        # Determine maximum number of victims within the same room
        # Double the actual size of maximum victims per room because player may stand at portal connecting two rooms
        self.max_victims_per_room = max(vg.size()) * 2
        
        for room_id in vg.groups.keys():
            victims = vg.get_group(room_id)
            room = rg.get_group(room_id)
            assert (victims['x'] <= room.reset_index().loc[0, 'x_max']).all() and \
                   (victims['x'] >= room.reset_index().loc[0, 'x_min']).all(), "Some victim in room {} is not within the room's X coordinate range!".format(room_id)
            assert (victims['z'] <= room.reset_index().loc[0, 'z_max']).all() and \
                   (victims['z'] >= room.reset_index().loc[0, 'z_min']).all(), "Some victim in room {} is not within the room's Y coordinate range!".format(room_id)
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 3: check_victim_in_room() completed. Elapsed time: {:.5f}".format(toc - tic))

        
    def _find_room_in(self):
        # Find the room that the player is currently in
        tic = time.perf_counter()
        
        # Util function for finding RoomID for each given player row consisting of x coordinate and z coordinate
        def find_room(in_row):
            x = in_row['x']
            z = in_row['z']
            result = 'None'
            for index, row in self.room_df.iterrows():
                if x >= row['x_min'] and x <= row['x_max'] and z >= row['z_min'] and z <= row['z_max']:
                    result = row['RoomID']
            return result
                
        # Add a new column to self.proc_df
        # Using parallelism if specified
        if using_parallel:
            self.proc_df["Room_in"] = self.proc_df[['x', 'z']].parallel_apply(find_room, axis=1)
        else:
            self.proc_df["Room_in"] = self.proc_df[['x', 'z']].apply(find_room, axis=1)
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 4: find_room_in() completed. Elapsed time: {:.5f}".format(toc - tic))
        
    def _find_portal_in(self):
        # Check if the victim is standing in the area of some portal
        tic = time.perf_counter()
        
        # Util function
        def find_portal(in_row):
            x, z = in_row['x'], in_row['z']
            result = 'None'
            for index, row in self.door_df.iterrows():
                if row['x0'] <= x <= row['x1'] and row['z0'] <= z <= row['z1']:
                    result = row['Index']
            return result
        
        if using_parallel:
            self.proc_df["Portal_in"] = self.proc_df[['x', 'z']].parallel_apply(find_portal, axis=1)
        else:
            self.proc_df["Portal_in"] = self.proc_df[['x', 'z']].apply(find_portal, axis=1)
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 4.5: find_portal_in() completed. Elapsed time: {:.5f}".format(toc - tic))
        
    def _find_victims_in_room(self):
        # Find the victims in the same room with the player
        # If player is standing in the area of a portal, will consider victims in both rooms the portal is connected to
        tic = time.perf_counter()
        
        vg = self.victim_df.groupby(by='RoomID')
        
        def find_victims(in_row):
            result = []
            
            # Enlist the rooms the player can observe. 2 rooms if player standing in portal, otherwise only 1 room
            if in_row['Portal_in'] != 'None':
                # Player standing in portal, grab both rooms
                portal_id = in_row['Portal_in']
                room_list = [self.door_df.loc[self.door_df['Index'] == portal_id]['Room0'].iloc[0], 
                             self.door_df.loc[self.door_df['Index'] == portal_id]['Room1'].iloc[0]]
            else:
                room_list = [in_row['Room_in']]
            
            # Find number of in-room victims
            num_victims_in_room = 0
            vic_index_list = []
            vic_color_list = []
            vic_dist_list = []
            vic_loc_list = []
            for room_id in room_list:
                num_vic_current_room = len(vg.get_group(room_id).index) if room_id in vg.groups.keys() else 0
                num_victims_in_room += num_vic_current_room
            
                # Find victim ids (Index in victim df) and corresponding colors
                vic_index_list += (list(vg.get_group(room_id)['Index'].to_numpy()) if room_id in vg.groups.keys() else [])
                vic_color_list += (list(vg.get_group(room_id)['Color'].to_numpy()) if room_id in vg.groups.keys() else [])
            
                # Calculate Euclidean distance to victim
                x_player = in_row['x']
                z_player = in_row['z']
            
                vic_dist_list += list(sqrt((vg.get_group(room_id).iloc[i]['x'] - x_player)**2 + (vg.get_group(room_id).iloc[i]['z'] - z_player)**2) for i in range(num_vic_current_room))
            
                vic_loc_list += list(tuple([vg.get_group(room_id).iloc[i]['x'], vg.get_group(room_id).iloc[i]['y'], vg.get_group(room_id).iloc[i]['z']]) for i in range(num_vic_current_room))
                
            # Form the result list
            result.append(num_victims_in_room)
            result += vic_index_list + ['None'] * (self.max_victims_per_room - num_victims_in_room)
            result += vic_color_list + ['None'] * (self.max_victims_per_room - num_victims_in_room)
            result += vic_dist_list + [0] * (self.max_victims_per_room - num_victims_in_room)
            result += vic_loc_list + [(0, 0, 0)] * (self.max_victims_per_room - num_victims_in_room)
            
            return result
        
        new_cols = ["num_victims"] + list("victim_{}_id".format(i) for i in range(self.max_victims_per_room)) \
                                   + list("victim_{}_color".format(i) for i in range(self.max_victims_per_room)) \
                                   + list("victim_{}_dist".format(i) for i in range(self.max_victims_per_room)) \
                                   + list("victim_{}_loc".format(i) for i in range(self.max_victims_per_room))
        
        # Using parallelism if specified
        if using_parallel:
            self.proc_df[new_cols] = self.proc_df[['Room_in', 'Portal_in', 'x', 'z']].parallel_apply(find_victims, axis=1, result_type='expand')
        else:
            self.proc_df[new_cols] = self.proc_df[['Room_in', 'Portal_in', 'x', 'z']].apply(find_victims, axis=1, result_type='expand')

        
        self.proc_df.drop(labels='index', axis=1, inplace=True)
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 5: find_victims_in_room() completed. Elapsed time: {:.5f}".format(toc - tic))
        
    def _set_role(self):
        tic = time.perf_counter()
        
        assert all(p in self.player_role.keys() for p in self.player_names), "unknown player {}".format(p)
        # Insert next to 'player_ID' column
        col_id = list(self.proc_df.columns).index('player_ID') + 1
        self.proc_df.insert(col_id, 'player_role', self.proc_df['player_ID'].apply(lambda n: self.player_role[n]))
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 6: set_role() completed. Elapsed time: {:.5f}".format(toc - tic))
        
    def _victim_in_FOV(self):
        # Determine if victims are in field of view of each player
        # Always false if the victim is not in the same room with the player
        tic = time.perf_counter()
        
        # Util function to be applied to each rows to generate new column
        def in_FOV(row):
            result = ['False', 'None', 'None', 'None'] * self.max_victims_per_room
            
            player_loc = tuple([row['x'], row['y'], row['z']])
            pitch, yaw = row['pitch'], row['yaw']
            for i in range(row['num_victims']):
                vic_loc = row['victim_{}_loc'.format(i)]
                result[4*i], result[4*i + 1], result[4*i + 2] = CanSee(player_loc, vic_loc, pitch, yaw)
                result[4*i] = str(result[4*i])
                # Calculate 2D angel degree
                result[4*i + 3] = np.sqrt(np.square(result[4*i + 1]) + np.square(result[4*i + 2]))
                
            return result

        new_cols = []
        for i in range(self.max_victims_per_room):
            new_cols += ['victim_{}_in_FOV'.format(i), 
                         'victim_{}_angel_yaw_deg'.format(i), 
                         'victim_{}_angel_pitch_deg'.format(i), 
                         'victim_{}_angel_2D_deg'.format(i)]
        self.proc_df[new_cols] = self.proc_df.apply(in_FOV, axis=1, result_type='expand')
        
        # Using debug mode of the FOV function and propagate debug results
#         def in_FOV_DEBUG(row):
#             result = ['False', 'None', 'None',] * self.max_victims_per_room
            
#             player_loc = tuple([row['x'], row['y'], row['z']])
#             pitch, yaw = row['pitch'], row['yaw']
#             for i in range(row['num_victims']):
#                 vic_loc = row['victim_{}_loc'.format(i)]
#                 tmp_result = CanSee(player_loc, vic_loc, pitch, yaw, debug=True)
#                 result[3*i], result[3*i + 1], result[3*i + 2] = str(tmp_result[0]), tmp_result[1], tmp_result[2]
                
#             return result
        
#         new_cols = []
#         for i in range(self.max_victims_per_room):
#             new_cols += ['victim_{}_in_FOV'.format(i), 'victim_{}_angle_yaw_deg'.format(i), 'victim_{}_angle_pitch_deg'.format(i)]
        
#         self.proc_df[new_cols] = self.proc_df.apply(in_FOV_DEBUG, axis=1, result_type='expand')
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 7: victim_in_FOV() completed. Elapsed time: {:.5f}".format(toc - tic))
        
        
    def _victim_in_triage_position(self):
        # Determine whether the victim is in FOV AND within max triage distance of the player
        # Only works for triager
        tic = time.perf_counter()
        
        # util function
        def in_TP(row):
            result = ['False'] * self.max_victims_per_room
            
            if row['player_role'] == 'Triager':
                for i in range(row['num_victims']):
                    result[i] = str(row['victim_{}_in_FOV'.format(i)] == 'True' and float(row['victim_{}_dist'.format(i)]) <= self.max_triage_dist)
                    
            return result
        
        new_cols = list('victim_{}_in_triage_pos'.format(i) for i in range(self.max_victims_per_room))
        self.proc_df[new_cols] = self.proc_df.apply(in_TP, axis=1, result_type='expand')
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 8: victim_in_triage_position() completed. Elapsed time: {:.5f}".format(toc - tic))
        
        
    def _victim_in_cross_hair(self):
        # Sets 'victim_in_crosshair_id' and 'victim_in_cross_hair_color'
        # Determine if victim is in the cross hair of each player. Criteria: victim in triage position & smallest angel_yaw_degree + angle_pitch_degree
        # Only consider True if player is a triager
        tic = time.perf_counter()
        
        # util function to be applied per row
        def in_CH(row):
            in_triage_pos = [row['victim_{}_in_triage_pos'.format(i)] == 'True' for i in range(self.max_victims_per_room)]
            angel_2D_deg = [row['victim_{}_angel_2D_deg'.format(i)] for i in range(self.max_victims_per_room)]
            CH_id = -1
            for i in range(self.max_victims_per_room):
                if not in_triage_pos[i]:
                    continue
                if CH_id < 0:
                    CH_id = i
                else:
                    if angel_2D_deg[i] < angel_2D_deg[CH_id]:
                        CH_id = i
            if CH_id < 0:
                results = ['None', 'None']
            else:
                results = [row['victim_{}_id'.format(CH_id)], row['victim_{}_color'.format(CH_id)]]
            return results
        
        new_cols = ['victim_in_crosshair_id', 'victim_in_cross_hair_color']
        self.proc_df[new_cols] = self.proc_df.apply(in_CH, axis=1, result_type='expand')
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 9: victim_in_cross_hair() completed. Elapsed time: {:.5f}".format(toc - tic))

        
    def _portal_in_FOV(self):
        # Sets 'portal_{}_in_FOV' with distance in ascending order, 'portal_{}_dist' and 'portal_{}_next_room'
        tic = time.perf_counter()
        
        # Util function to be applied to each row of proc_df
        def info_doors_in_FOV(row):
            # Return max_portal_in_FOV number of groups of results. 
            #   Each group contains 3 fields: 'door_{}_in_FOV', 'portal_{}_dist', and 'portal_{}_next_room'. 
            #   The groups are sorted in ascending order based on distance
            result = ['None', 0, 'None',] * self.max_portal_in_FOV

            player_loc = tuple([row['x'], row['y'], row['z']])
            player_y = row['y']
            pitch, yaw = row['pitch'], row['yaw']
            room = row['Room_in']

            # Util function to be applied to each row of door_df
            def in_FOV(row):
                # Check if the door is in the same room as the player
                if room != row['Room0'] and room != row['Room1']:
                    return [False, 0]

                door_loc = tuple([row['x_mid'], player_y, row['z_mid']])
                can_see, _, _ = CanSee(player_loc, door_loc, pitch, yaw)
                horiz_dist = math.sqrt(math.pow(door_loc[0] - player_loc[0], 2) + math.pow(door_loc[2] - player_loc[2], 2))

                return [can_see, (horiz_dist if can_see else 0)]

            info = self.door_df.apply(in_FOV, axis=1, result_type='expand')
            info.columns = ['labels', 'dist']
            portal_group = pd.concat([self.door_df, info], axis=1).groupby(by='labels')

            if True in portal_group.groups.keys():
                sorted_portals = portal_group.get_group(True).sort_values(by='dist', axis=0, ascending=True)
                sorted_portals = sorted_portals.reset_index().drop('index', axis=1)
                sorted_portals = sorted_portals.iloc[:self.max_portal_in_FOV]
                for index, row in sorted_portals.iterrows(): 
                    result[3*index], result[3*index + 1], result[3*index + 2] = row['Index'], row['dist'], row['Room0'] if row['Room0'] != room else row['Room1']

            return result
        
        new_cols = []
        for i in range(self.max_portal_in_FOV):
            new_cols += ['portal_{}_in_FOV'.format(i), 'portal_{}_dist'.format(i), 'portal_{}_next_room'.format(i)]
        self.proc_df[new_cols] = self.proc_df.apply(info_doors_in_FOV, axis=1, result_type='expand')
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 10: portal_in_FOV() completed. {:.5f}".format(toc - tic))
        
    
    def _identify_triage_id(self):
        # Sets 'victim_id' and 'proc_df_id' field in triage dataframe
        # Identify the victim id corresponding to each of the triaging event
        tic = time.perf_counter()
        
        self.triage_df[['victim_id', 'victim_color']] = self.triage_df.apply(lambda row: 
                                                           self.victim_df.loc[(self.victim_df['x'] == row['victim_x']) &
                                                                              (self.victim_df['y'] == row['victim_y']) &
                                                                              (self.victim_df['z'] == row['victim_z'])][['Index', 'Color']].iloc[0], 
                                                           axis=1)
        # Identify index of row entry in the main datafram proc_df_id that has closest timestamp to each entry in triage df
        # Util function for finding index of entry:
        def find_id(row):
            abs_diff = (pd.to_datetime(self.proc_df.loc[self.proc_df['player_ID'] == row['player_ID']]['@timestamp']) 
                                                         - pd.to_datetime(row['@timestamp'])).abs()
            pos = abs_diff.argsort().iloc[0]
            index = abs_diff.index[pos]
            return index
        
        # Using parallelism if specified
        if using_parallel:
            self.triage_df['proc_df_id'] = self.triage_df.parallel_apply(find_id, axis=1)
        else:
            self.triage_df['proc_df_id'] = self.triage_df.apply(find_id, axis=1)

        # Drop duplicates
        self.triage_df = self.triage_df.drop_duplicates(subset=self.triage_df.columns.drop('@timestamp'), keep='last')
        self.triage_df = self.triage_df.reset_index().drop('index', axis=1)
        
        # Check that victim id agrees in triage df and in 'crosshair' field in proc_df
        combined_col_labels = list(self.proc_df.columns) + ['triage_state', 'victim_id', 'proc_df_id']
        self.conflict_victim_df = pd.DataFrame(columns=combined_col_labels)
        for index, row in self.triage_df.iterrows():
            if not row['victim_id'] == self.proc_df['victim_in_crosshair_id'].iloc[row['proc_df_id']]:
                if self.verbose:
                    warn("Found conflict of victim entries. Found victim_id '{}' at index '{}' in triage_df, whereas victim_id '{}' in the crosshair " \
                         "field at index '{}' in proc_df \n\tvictim"\
                         .format(row['victim_id'], index, self.proc_df['victim_in_crosshair_id'].iloc[row['proc_df_id']], row['proc_df_id']))
                combined_row = self.proc_df.iloc[row['proc_df_id']].append(row[['triage_state', 'victim_id', 'proc_df_id']])
                self.conflict_victim_df = self.conflict_victim_df.append(combined_row, ignore_index=True)
        
        new_col_labels = list(self.proc_df.columns) + ['event_triage_state', 'event_victim_id', 'proc_df_id']
        self.conflict_victim_df.columns = new_col_labels
        
        if self.conflict_victim_df.shape[0] > 0:
            # Map player locations to original values without offset, rounded to integer
            self.conflict_victim_df[['x', 'y', 'z']] = self.conflict_victim_df.apply(lambda row: [row['x'] - self.x_offset, row['y'] - self.y_offset, row['z'] - self.z_offset], 
                                                                                     axis=1, result_type = 'expand')
            # Transpose the dataframe for better visual
            self.conflict_victim_df = self.conflict_victim_df.transpose()

            path = os.path.join(os.getcwd(), "conflict_victims")
            if not os.path.exists(path):
                os.makedirs(path)
            conflict_filename = "conflct_{}.csv".format(datetime.now().strftime("%m-%dT%H-%M-%S"))
            self.conflict_victim_df.to_csv(os.path.join(path, conflict_filename))

#             if self.verbose:
            print("\nConflict entries saved to {}\n".format(os.path.join(path, conflict_filename)))
        else:
            print("\nCONGRATULATIONS No conflict entries found!\n")
        
        # Publish the victim id to proc_df
        self.proc_df['event_triage_victim_id'] = 'None'
        for index, row in self.triage_df.iterrows():
            self.proc_df['event_triage_victim_id'].iloc[row['proc_df_id']] = row['victim_id']
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 11: identify_triage_id() completed. {:.5f}".format(toc - tic))
    
    
    def _set_triage_in_progress(self):
        # Sets 'triage_in_progress' and 'triage_result' field
        # Based on event df, determine if the player is triaging the victim that is in its crosshair
        tic = time.perf_counter()
        
        # First add new columns and set with default value 'False' to 'triage_in_progress' and 'None' to 'triage_result'
        self.proc_df['triage_in_progress'] = 'False'
        self.proc_df['triage_result'] = 'None'
        
        # Group triage df by the triage state. Get the group where state is IN_PROGRESS. Iterate through rows of this group
        in_prog_group = self.triage_df.groupby(by='triage_state').get_group("IN_PROGRESS")
        for index, row in in_prog_group.iterrows():
            if index == self.triage_df.shape[0] - 1 and self.verbose:
                warn("\n Found a 'Triage In Progress' event as the last triage event with no result at index {} of the " 
                     "triage dataframe. This event will NOT be registered in the main dataframe.".format(index))
            # Find the result of this triage episode by looking up next row in triage_df
            next_row = self.triage_df.iloc[index + 1]
            proc_df_id_1 = row['proc_df_id']
            proc_df_id_2, result, victim_id = next_row['proc_df_id'], next_row['triage_state'], next_row['victim_id']
            
            if result not in ['SUCCESSFUL', 'UNSUCCESSFUL'] and self.verbose:
                warn("\nExpect 'SUCCESSFUL' or 'UNSUCCESSFUL' triage state in the next row entry of triage dataframe, "
                     "but found {} at row index {}".format(result, index + 1))
            
            # Backfill data
            self.proc_df['triage_in_progress'].iloc[proc_df_id_1:proc_df_id_2 + 1] = 'True'
            self.proc_df['triage_result'].iloc[proc_df_id_1:proc_df_id_2 + 1] = result
            self.proc_df['event_triage_victim_id'].iloc[proc_df_id_1:proc_df_id_2 + 1] = victim_id
            
        toc = time.perf_counter()
        if self.verbose:
            print("Step 12: set_triage_in_progress() completed. {:.5f}".format(toc - tic))
            
    def _update_victim_color(self):
        # Update the victim color to White once they are successfully triaged
        tic = time.perf_counter()
        
        # Util function to be appied to triage_df row by row
        def update_by_row(row):
            if row['triage_state'] != 'SUCCESSFUL':
                return
            vic_id = row['victim_id']
            proc_df_id = row['proc_df_id']
            # Update 'victim_{}_color' field
            for pos in range(self.max_victims_per_room):
                self.proc_df.loc[(self.proc_df['victim_{}_id'.format(pos)] == vic_id) & 
                                 (self.proc_df.index >= proc_df_id), 'victim_{}_color'.format(pos)] = 'White'
            # Update 'victim_in_cross_hair_color' field
            self.proc_df.loc[(self.proc_df['victim_in_crosshair_id'] == vic_id) &
                             (self.proc_df.index >= proc_df_id), 'victim_in_crosshair_color'] = 'White'

        # Apply function to triage_df
        self.triage_df.apply(update_by_row, axis=1)    
        
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 13: update_victim_color() completed. {:.5f}".format(toc - tic))
        
        
    def _set_sensor_state(self):
        # Iterate through all event identifiers and find the occurrence of each in the event dataframe
        # If found event occurence, find the closest row entry in proc_df dataframe in time and set value of corresponding 
        #     column 
        tic = time.perf_counter()
        
        # Add sensor state column
        self.proc_df['sensor_state'] = 'None'
        
        # Util function that sets field value of proc_df row that is closet in time
        def find_id(row):
            abs_diff = (pd.to_datetime(self.proc_df['@timestamp']) - pd.to_datetime(row['@timestamp'])).abs()
            pos = abs_diff.argsort().iloc[0]
            index = abs_diff.index[pos]
            
            if self.proc_df['sensor_state'].iloc[index] == 'None':
                self.proc_df['sensor_state'].iloc[index] = row['subtype']
            elif self.proc_df['sensor_state'].iloc[index] != row['subtype']:
                self.proc_df['sensor_state'].iloc[index] += "," + row['subtype']
        
        # Pick sensor beeping events
        for event_str in self.sensor_events:
            sensor_df = self.event_df.loc[self.event_df['subtype'] == event_str]
            sensor_df.apply(find_id, axis=1)
        
        toc = time.perf_counter()
        if self.verbose:
            print("Step 14: set_sensor_state() completed. {:.5f}".format(toc - tic))
        

## CSV Data

In [95]:
# File locations and other parameters
main_data_path = "/mnt/sda2/projects/ASIST/ASIST-Sigma/data2/8-13/raw"
env_data_path = "/mnt/sda2/projects/ASIST/ASIST-Sigma/data2/falcon"

event_obs_file_name = "20200810_Participant9_Cond1_1of2.csv"

room_csv_name = "ASIST_FalconMap_Rooms_v1.1_OCN.csv"
door_csv_name = "ASIST_FalconMap_Portals_v1.1_OCN.csv"
victim_csv_name = "ASIST_FalconMap_Victims_v1.1_OCN.csv"



In [96]:
self = DataLoader()

In [97]:
# Load main data file
self.load_csv(main_data_path, event_obs_file_name)

# Load map data, including room, door, and victim locations
self.load_map(env_data_path, room_csv_name, door_csv_name, victim_csv_name)

# Set frequency -- number of entries per second. Will be spreaded evenly in time
self.set_num_entries_per_sec(2)       # Set to select 2 entries per second. Divided equally in time

# Sets sensor event subtype string
self.add_sensor_event_subtype('Event:Beep')
self.add_sensor_event_subtype('Event:Woof')

/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:111: UserWarning: 
No 'mission_state == STOP' event found. Entries late in time will not be chopped off



ValueError: Columns must be same length as key

In [99]:
self.triage_df

,message,content,content_keys,@timestamp,message_type,msg,data,subtype


In [61]:
# Process
self.process()

Step 1: df_by_sec() completed. Elapsed time: 0.01166
Step 2: adjust_loc() completed. Elapsed time: 0.02068
Step 3: check_victim_in_room() completed. Elapsed time: 0.04681
Step 4: find_room_in() completed. Elapsed time: 8.86072
Step 4.5: find_portal_in() completed. Elapsed time: 9.03647
Step 5: find_victims_in_room() completed. Elapsed time: 3.21897
Step 6: set_role() completed. Elapsed time: 0.00091
Step 7: victim_in_FOV() completed. Elapsed time: 0.38636
Step 8: victim_in_triage_position() completed. Elapsed time: 0.19237
Step 9: victim_in_cross_hair() completed. Elapsed time: 0.30336
Step 10: portal_in_FOV() completed. 19.04947


/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:669: UserWarning: Found conflict of victim entries. Found victim_id '24' at index '68' in triage_df, whereas victim_id '25' in the crosshair field at index '925' in proc_df 
	victim
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:669: UserWarning: Found conflict of victim entries. Found victim_id '20' at index '81' in triage_df, whereas victim_id '19' in the crosshair field at index '1151' in proc_df 
	victim
/home/jincheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)



Conflict entries saved to /mnt/sda2/projects/ASIST/ASIST-Sigma/data2/conflict_victims/conflct_08-18T16-51-53.csv

Step 11: identify_triage_id() completed. 2.98014


/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:726: UserWarning: 
Expect 'SUCCESSFUL' or 'UNSUCCESSFUL' triage state in the next row entry of triage dataframe, but found IN_PROGRESS at row index 3
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:726: UserWarning: 
Expect 'SUCCESSFUL' or 'UNSUCCESSFUL' triage state in the next row entry of triage dataframe, but found IN_PROGRESS at row index 8
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:726: UserWarning: 
Expect 'SUCCESSFUL' or 'UNSUCCESSFUL' triage state in the next row entry of triage dataframe, but found IN_PROGRESS at row index 13
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:726: UserWarning: 
Expect 'SUCCESSFUL' or 'UNSUCCESSFUL' triage state in the next row entry of triage dataframe, but found IN_PROGRESS at row index 18
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:726: UserWarning: 
Expe

Step 12: set_triage_in_progress() completed. 4.70666
Step 13: update_victim_color() completed. 0.25854


/home/jincheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Step 14: set_sensor_state() completed. 0.53216


In [62]:
save_file_name_prefix = "processed_"
self.save_csv(os.path.join(main_data_path, 'processed'), save_file_name_prefix + os.path.splitext(event_obs_file_name)[0] + '.csv')

# JSON Data

In [33]:
# Sparky
main_data_path = "/mnt/sda2/projects/ASIST/ASIST-Sigma/data2/6-22/Sparky"
env_data_path = "/mnt/sda2/projects/ASIST/ASIST-Sigma/data2/sparky"

event_obs_file_name = "ASIST_data_study_id_000001_condition_id_000003_trial_id_000013_messages.json"

room_csv_name = "ASIST_SparkyMap_Rooms_v5.3_OCN.csv"
door_csv_name = "ASIST_SparkyMap_Portals_v5.4_OCN.csv"
victim_csv_name = "ASIST_SparkyMap_Victims_v5.2_OCN.csv"


In [48]:
# Falcon 
main_data_path = "/mnt/sda2/projects/ASIST/ASIST-Sigma/data2/6-22/Falcon"
env_data_path = "/mnt/sda2/projects/ASIST/ASIST-Sigma/data2/falcon"

event_obs_file_name = "ASIST_data_study_id_000001_condition_id_000007_trial_id_000012_messages.json"

room_csv_name = "ASIST_FalconMap_Rooms_v1.1_OCN.csv"
door_csv_name = "ASIST_FalconMap_Portals_v1.1_OCN.csv"
victim_csv_name = "ASIST_FalconMap_Victims_v1.1_OCN.csv"

In [49]:
self = DataLoader()

In [ ]:
# Load main data file
self.load_json(main_data_path, event_obs_file_name)

# Load map data, including room, door, and victim locations
self.load_map(env_data_path, room_csv_name, door_csv_name, victim_csv_name)

# Set frequency -- number of entries per second. Will be spreaded evenly in time
self.set_num_entries_per_sec(2)       # Set to select 2 entries per second. Divided equally in time

# Sets sensor event subtype string
self.add_sensor_event_subtype('Event:Beep')
self.add_sensor_event_subtype('Event:Woof')

In [160]:
self.process()

Step 1: df_by_sec() completed. Elapsed time: 0.01060
Step 2: adjust_loc() completed. Elapsed time: 0.01971
Step 3: check_victim_in_room() completed. Elapsed time: 0.04950
Step 4: find_room_in() completed. Elapsed time: 7.97832
Step 4.5: find_portal_in() completed. Elapsed time: 8.10604
Step 5: find_victims_in_room() completed. Elapsed time: 3.47323
Step 6: set_role() completed. Elapsed time: 0.00067
Step 7: victim_in_FOV() completed. Elapsed time: 0.37167
Step 8: victim_in_triage_position() completed. Elapsed time: 0.17707
Step 9: victim_in_cross_hair() completed. Elapsed time: 0.26600
Step 10: portal_in_FOV() completed. 16.96075


/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:664: UserWarning: Found conflict of victim entries. Found victim_id '3' at index '4' in triage_df, whereas victim_id 'None' in the crosshair field at index '342' in proc_df 
	victim
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:664: UserWarning: Found conflict of victim entries. Found victim_id '3' at index '5' in triage_df, whereas victim_id 'None' in the crosshair field at index '372' in proc_df 
	victim
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:664: UserWarning: Found conflict of victim entries. Found victim_id '19' at index '12' in triage_df, whereas victim_id '20' in the crosshair field at index '585' in proc_df 
	victim
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:664: UserWarning: Found conflict of victim entries. Found victim_id '19' at index '13' in triage_df, whereas victim_id '20' in the crosshair field at inde


Conflict entries saved to /mnt/sda2/projects/ASIST/ASIST-Sigma/data2/conflict_victims/conflct_07-22T17-04-27.csv

Step 11: identify_triage_id() completed. 2.02850
Step 12: set_triage_in_progress() completed. 2.56589
Step 13: update_victim_color() completed. 0.37482


/home/jincheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Step 14: set_sensor_state() completed. 1.18201


In [38]:
save_file_name_prefix = "processed_"
self.save_csv(main_data_path, save_file_name_prefix + os.path.splitext(event_obs_file_name)[0] + '.csv')

# Batch Processing JSON Data

In [ ]:
main_data_path = "/media/jincheng/Shared/projects/ASIST/ASIST-Sigma/data2/6-22"
env_data_path = "/media/jincheng/Shared/projects/ASIST/ASIST-Sigma/data2"

room_csv_name = "DARPA-ASIST_ICT_MapData - RoomLocations_v5.3.csv"
door_csv_name = "DARPA-ASIST_ICT_MapData - Door locations_v5.3.csv"
victim_csv_name = "DARPA-ASIST_ICT_MapData - VictimLocations_v5.2.csv"

In [ ]:
for file_name in os.listdir(main_data_path):
    if file_name.endswith(".json"):
        try:
            print("\n\n---------------- Processing file {} ----------------\n".format(file_name))
            self = DataLoader()
            
            # Load main data file
            self.load_json(main_data_path, file_name)

            # Load map data, including room, door, and victim locations
            self.load_map(env_data_path, room_csv_name, door_csv_name, victim_csv_name)

            # Set coordinate offsets
            self.set_loc_offsets(2202, -23, -144)

            # Set time span (number of seconds) necessary to successfully triage each type of victim
            # Round up to specify a coarse upper bound
            victim_time = {'Green': 8, 'Gold': 16}
            self.set_victim_triage_time_span(victim_time)

            # Set frequency -- number of entries per second. Will be spreaded evenly in time
            self.set_num_entries_per_sec(2)       # Set to select 2 entries per second. Divided equally in time

            # Sets sensor event subtype string
            self.add_sensor_event_subtype('Event:Beep')
            self.add_sensor_event_subtype('Event:Woof')
            
            self.process()
            
            self.save_csv(os.path.join(main_data_path, 'processed'), "processed_" + os.path.splitext(file_name)[0] + '.csv')
            print("\n---------------- File saved ----------------\n\n")
        except Exception as err:
            warn("CANNOT PROCESS JSON FILE: {}. Encountered the following exception: \n\n{}".format(file_name, err))


# Multiprocessing Batch JSON Data

In [6]:
# Sparky map
env_data_path = "/mnt/sda2/projects/ASIST/ASIST-Sigma/data2/sparky"

room_csv_name = "ASIST_SparkyMap_Rooms_v5.3_OCN.csv"
door_csv_name = "ASIST_SparkyMap_Portals_v5.4_OCN.csv"
victim_csv_name = "ASIST_SparkyMap_Victims_v5.2_OCN.csv"

# Main data path
main_data_path = "/mnt/sda2/projects/ASIST/ASIST-Sigma/data2/6-22/Sparky"

In [100]:
# Falcon map 
env_data_path = "/mnt/sda2/projects/ASIST/ASIST-Sigma/data2/falcon"
room_csv_name = "ASIST_FalconMap_Rooms_v1.1_OCN.csv"
door_csv_name = "ASIST_FalconMap_Portals_v1.1_OCN.csv"
victim_csv_name = "ASIST_FalconMap_Victims_v1.1_OCN.csv"

# Main data path
main_data_path = "/mnt/sda2/projects/ASIST/ASIST-Sigma/data2/8-13/raw"

In [101]:
verbose = True
file_format = "csv"   # "csv" or "json"

def proc(file_name):
    try:
        print("\n\n---------------- Processing file {} ----------------\n".format(file_name))
        self = DataLoader(verbose=verbose)

        # Load main data file
        if file_format == "csv":
            self.load_csv(main_data_path, file_name)
        else:
            self.load_json(main_data_path, file_name)

        # Load map data, including room, door, and victim locations
        self.load_map(env_data_path, room_csv_name, door_csv_name, victim_csv_name)

        # Set frequency -- number of entries per second. Will be spreaded evenly in time
        self.set_num_entries_per_sec(2)       # Set to select 2 entries per second. Divided equally in time

        # Sets sensor event subtype string
        self.add_sensor_event_subtype('Event:Beep')
        self.add_sensor_event_subtype('Event:Woof')

        self.process()

        save_file_name = "processed_" + os.path.splitext(file_name)[0] + '.csv'
        self.save_csv(os.path.join(main_data_path, 'processed'), save_file_name)
        print("\n---------------- File saved: {} ----------------\n\n".format(save_file_name))
    except Exception as err:
        warn("CANNOT PROCESS JSON FILE: {}. Encountered the following exception: \n\n{}".format(file_name, err))

In [102]:
# Using a pool of workers
from multiprocessing import Pool

with Pool() as pool:
    # Gather all JSON files in the given directory
    file_list = list(file_name for file_name in os.listdir(main_data_path) if file_name.endswith(file_format))
    # Map to processes
    pool.map(proc, file_list)
    print("### ALL FINISHED! ###")



---------------- Processing file 20200728_Participant3_Cond1_timestamp.csv ----------------


---------------- Processing file 20200818_Participant16_Cond1.csv ----------------


---------------- Processing file 20200817_Participant14_Cond1.csv ----------------


---------------- Processing file 20200818_Participant17_Cond1.csv ----------------


---------------- Processing file 20200811_Participant11_Cond1.csv ----------------


---------------- Processing file 20200728_Participant3_Cond1.csv ----------------


---------------- Processing file 20200724_Participant1_Cond1_X.csv ----------------


---------------- Processing file 20200729_Participant4_Cond1.csv ----------------


---------------- Processing file 20200805_Participant7_Cond1.csv ----------------



---------------- Processing file 20200813_Participant12_Cond1.csv ----------------





---------------- Processing file 20200810_Participant9_Cond1_2of2.csv ----------------



---------------- Processing file 20200807_Parti

/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: UserWarning: 
No Event:MissionState event found! All raw data entries are therefore taken valid

/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:111: UserWarning: 
No 'mission_state == STOP' eve

Step 1: df_by_sec() completed. Elapsed time: 0.00862
Step 2: adjust_loc() completed. Elapsed time: 0.03959
Step 1: df_by_sec() completed. Elapsed time: 0.01231
Step 3: check_victim_in_room() completed. Elapsed time: 0.08762
Step 2: adjust_loc() completed. Elapsed time: 0.02633
Step 3: check_victim_in_room() completed. Elapsed time: 0.07053
Step 1: df_by_sec() completed. Elapsed time: 0.01190
Step 2: adjust_loc() completed. Elapsed time: 0.02671
Step 1: df_by_sec() completed. Elapsed time: 0.01680
Step 2: adjust_loc() completed. Elapsed time: 0.02933
Step 3: check_victim_in_room() completed. Elapsed time: 0.07117
Step 3: check_victim_in_room() completed. Elapsed time: 0.07073
Step 1: df_by_sec() completed. Elapsed time: 0.01622
Step 2: adjust_loc() completed. Elapsed time: 0.03341
Step 3: check_victim_in_room() completed. Elapsed time: 0.07118
Step 1: df_by_sec() completed. Elapsed time: 0.01699
Step 2: adjust_loc() completed. Elapsed time: 0.04555
Step 3: check_victim_in_room() complet

/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:671: UserWarning: Found conflict of victim entries. Found victim_id '5' at index '2' in triage_df, whereas victim_id '3' in the crosshair field at index '51' in proc_df 
	victim
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:671: UserWarning: Found conflict of victim entries. Found victim_id '3' at index '6' in triage_df, whereas victim_id '0' in the crosshair field at index '81' in proc_df 
	victim
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:671: UserWarning: Found conflict of victim entries. Found victim_id '0' at index '10' in triage_df, whereas victim_id '2' in the crosshair field at index '107' in proc_df 
	victim



Conflict entries saved to /mnt/sda2/projects/ASIST/ASIST-Sigma/data2/conflict_victims/conflct_08-18T17-11-16.csv

Step 4: find_room_in() completed. Elapsed time: 5.86481


...

Step 14: set_sensor_state() completed. 7.43147

---------------- File saved: processed_20200731_Participant5_Cond1.csv ----------------


Step 14: set_sensor_state() completed. 14.70926

---------------- File saved: processed_20200717_Participant0_MinecraftExperiment.csv ----------------




/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:728: UserWarning: 
Expect 'SUCCESSFUL' or 'UNSUCCESSFUL' triage state in the next row entry of triage dataframe, but found IN_PROGRESS at row index 8
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:728: UserWarning: 
Expect 'SUCCESSFUL' or 'UNSUCCESSFUL' triage state in the next row entry of triage dataframe, but found IN_PROGRESS at row index 14
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:728: UserWarning: 
Expect 'SUCCESSFUL' or 'UNSUCCESSFUL' triage state in the next row entry of triage dataframe, but found IN_PROGRESS at row index 19
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:728: UserWarning: 
Expect 'SUCCESSFUL' or 'UNSUCCESSFUL' triage state in the next row entry of triage dataframe, but found IN_PROGRESS at row index 24
/home/jincheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:728: UserWarning: 
Exp

Step 12: set_triage_in_progress() completed. 17.85880
Step 13: update_victim_color() completed. 0.21640


/home/jincheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Step 14: set_sensor_state() completed. 5.14771

---------------- File saved: processed_20200724_Participant1_Cond1.csv ----------------


### ALL FINISHED! ###


# Temporary

In [ ]:
self.proc_df.columns